In [1]:
import json

In [23]:
trainlol = json.load(open("/scratch/tgokhale/VQA_LOL_ECCV/train_vqa_lol_varlen.json"))

In [24]:
traincwl = json.load(open("/scratch/tgokhale/VQA_LOL_ECCV/train_vqa_objcap_varlen.json"))

In [26]:
train = trainlol+traincwl

In [25]:
vallol = json.load(open("/scratch/tgokhale/VQA_LOL_ECCV/minival_vqa_lol_varlen.json"))
valcwl = json.load(open("/scratch/tgokhale/VQA_LOL_ECCV/minival_vqa_lol_varlen.json"))

In [27]:
val = vallol+valcwl

In [28]:
json.dump(train,open("data/train.json","w"))

In [29]:
json.dump(val,open("data/val.json","w"))

In [30]:
item = val[100]

In [31]:
item

{'answer_type': 'yes/no',
 'img_id': 'COCO_val2014_000000132136',
 'label': {'no': 1},
 'question_id': 132136015,
 'question_type': 'is it',
 'sent': 'Is this the beach and Are these balloons and Are not these balloons  ?',
 'q_list': ['Is this the beach', 'Are these balloons', 'Are these balloons'],
 'q_label_list': [{'yes': 1}, {'no': 1}, {'no': 1}],
 'r_list': ['Is this the beach',
  'Are these balloons',
  'Are not these balloons '],
 'r_label_list': [{'yes': 1}, {'no': 1}, {'yes': 1}],
 'n': 3,
 'op': 'and'}

In [18]:
def get_matched(answer_tokens,ind,words_tokens):
    matched = False
    curr_match = False
    curr_ans_len = -1
    for ans_i in range(len(answer_tokens)):
        curr_match = False
        curr_ans_len = len(answer_tokens[ans_i])
        for token_A,token_B in zip(answer_tokens[ans_i],words_tokens[ind:ind+curr_ans_len]):
            if token_A==token_B:
                curr_match=True
            else:
                curr_match=False
                break
        if curr_match:
            matched=True
            break
    return curr_match,curr_ans_len

def generate_bio(text,answers):
    words_tokens = text.lower().split(' ')
    answer_tokens = [ answer.lower().split(' ') for answer in answers]
    tagged_tokens = []
    ind = 0
    while len(tagged_tokens) <= len(words_tokens):
        matched,len_ans = get_matched(answer_tokens,ind,words_tokens)
        if matched:
            tagged_tokens.append('B-ANS')
            num_of_is = ['I-ANS']*(len_ans-1)
            tagged_tokens.extend(num_of_is)
            ind += len_ans
        else:
            ind+=1
            tagged_tokens.append('O')
    tagged_tokens.pop(-1)
    if len(words_tokens) != len(tagged_tokens):
        print(text,words_tokens,tagged_tokens,len(words_tokens)-len(tagged_tokens))
    return tagged_tokens

In [33]:
text = item['sent']
text = text.replace("?"," ?")
generate_bio(text,item["r_list"])

['B-ANS',
 'I-ANS',
 'I-ANS',
 'I-ANS',
 'O',
 'B-ANS',
 'I-ANS',
 'I-ANS',
 'O',
 'B-ANS',
 'I-ANS',
 'I-ANS',
 'I-ANS',
 'I-ANS',
 'O',
 'O']